# Interactive visualisation of the prediction for a single subject

In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

from prediction.mrs_outcome_prediction.LSTM.testing.shap_helper_functions import check_shap_version_compatibility
from prediction.mrs_outcome_prediction.data_loading.data_formatting import format_to_2d_table_with_time, \
    feature_order_verification
from prediction.utils.utils import check_data
from preprocessing.preprocessing_tools.normalisation.reverse_normalisation import reverse_normalisation
from prediction.mrs_outcome_prediction.LSTM.testing.visualisation_helper_functions import reverse_normalisation_for_subj
from sklearn.model_selection import train_test_split
from prediction.mrs_outcome_prediction.data_loading.data_formatting import features_to_numpy, \
    link_patient_id_to_outcome, numpy_to_lookup_table

In [ ]:
# Shap values require very specific versions
check_shap_version_compatibility()

### Pre-computation

*This takes some time to compute and should be run in advance*

In [ ]:
model_weights_path = '/Users/jk1/temp/mimic/prediction/3M_Death/test_LSTM_sigmoid_all_balanced_0.6_1_True_RMSprop_3M Death_8_3/sigmoid_all_balanced_0.6_1_True_RMSprop_3M Death_8_3.hdf5'

training_features_path = '/Users/jk1/temp/opsum_prepro_output/preprocessed_features_27092022_172403.csv'
training_labels_path = '/Users/jk1/temp/opsum_prepro_output/preprocessed_outcomes_27092022_172403.csv'

validation_features_path = '/Users/jk1/temp/mimic/preprocessing/mimic_prepro_30122022_133144/preprocessed_features_30122022_133144.csv'
validation_labels_path = '/Users/jk1/temp/mimic/preprocessing/mimic_prepro_30122022_133144/preprocessed_outcomes_30122022_133144.csv'

normalisation_parameters_path = '/Users/jk1/temp/mimic/preprocessing/mimic_prepro_30122022_133144/logs_30122022_133144/reference_population_normalisation_parameters.csv'
shap_over_time_path = '/Users/jk1/temp/mimic/prediction/3M_Death/test_LSTM_sigmoid_all_balanced_0.6_1_True_RMSprop_3M Death_8_3/deep_explainer_shap_values_over_ts.pkl'
predictions_over_time_path = '/Users/jk1/temp/mimic/prediction/3M_Death/test_LSTM_sigmoid_all_balanced_0.6_1_True_RMSprop_3M Death_8_3/predictions_over_timesteps.pkl'
out_dir = '/Users/jk1/Downloads'

In [ ]:
save_figures = False

In [ ]:
outcome = '3M Death'

model_name = os.path.basename(model_weights_path).split('.hdf5')[0]

model_config = {
    'activation': model_name.split('_')[0],
    'batch': model_name.split('_')[1],
    'data': model_name.split('_')[2],
    'dropout': float(model_name.split('_')[3]),
    'layers': int(model_name.split('_')[4]),
    'masking': model_name.split('_')[5],
    'optimizer': model_name.split('_')[6],
    'units': int(model_name.split('_')[8]),
    'cv_fold': int(model_name.split('_')[9]),
    'seed': 42,
    'test_size': 0.20
}

override_masking_value = False

In [ ]:
n_features = 3

In [ ]:
with open(shap_over_time_path, 'rb') as handle:
    shap_values_over_time = pickle.load(handle)

normalisation_parameters_df = pd.read_csv(normalisation_parameters_path)


In [ ]:
with open(predictions_over_time_path, 'rb') as handle:
    predictions_over_time = pickle.load(handle)

### Load the training data

In [ ]:
## LOAD THE TRAINING DATASET
derivation_X, derivation_y = format_to_2d_table_with_time(feature_df_path=training_features_path, outcome_df_path=training_labels_path,
                                    outcome=outcome)

check_data(derivation_X)

# Reduce every patient to a single outcome (to avoid duplicates)
all_pids_with_outcome = link_patient_id_to_outcome(derivation_y, outcome)
pid_train, pid_test, _, _ = train_test_split(all_pids_with_outcome.patient_id.tolist(),
                                                all_pids_with_outcome.outcome.tolist(),
                                                stratify=all_pids_with_outcome.outcome.tolist(),
                                                test_size=model_config['test_size'],
                                                random_state=model_config['seed'])
train_X_df = derivation_X[derivation_X.patient_id.isin(pid_train)]
train_X_np = features_to_numpy(train_X_df,
                               ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
feature_order_verification(train_X_np)
# Remove the case_admission_id, sample_label, and time_step_label columns from the data
train_X_np = train_X_np[:, :, :, -1].astype('float32')


### Load validation data

In [ ]:
# load the dataset
X, y = format_to_2d_table_with_time(feature_df_path=validation_features_path, outcome_df_path=validation_labels_path,
                                    outcome=outcome)

n_time_steps = X.relative_sample_date_hourly_cat.max() + 1
n_channels = X.sample_label.unique().shape[0]
# test if data is corrupted
check_data(X)
test_X_np = features_to_numpy(X,
                              ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
# ensure that the order of features (3rd dimension) is the one predefined for the model
feature_order_verification(test_X_np)
# create look-up table for case_admission_ids, sample_labels and relative_sample_date_hourly_cat
test_features_lookup_table = numpy_to_lookup_table(test_X_np)
test_y_np = np.array([y[y.case_admission_id == cid].outcome.values[0] for cid in
                      test_X_np[:, 0, 0, 0]]).astype('float32')
# Remove the case_admission_id, sample_label, and time_step_label columns from the data
test_X_np = test_X_np[:, :, :, -1].astype('float32')

In [ ]:
features = list(test_features_lookup_table['sample_label'].keys())

In [ ]:
non_normalised_train_X_df = reverse_normalisation(train_X_df, normalisation_parameters_df)

## Choose subject and load or compute prediction

In [ ]:
subj = randint(0, len(test_X_np))

In [ ]:
print(f'Subject index: {subj}')
print(f'Overall {outcome} prediction: {predictions_over_time[-1, subj] * 100:.2f}%')
print(f'Final outcome: {outcome} - {bool(test_y_np[subj])}')

In [ ]:
subj_pred_over_ts = predictions_over_time[:,subj]

## Plot overall subject prediction & explanation

In [ ]:
n_features = 3

In [ ]:
# plot a bar plot showing impact of most important features on the prediction across all n_time_steps
# find index of 3 features with biggest positive shap impart
selected_positive_features = np.squeeze(shap_values_over_time[-1])[subj].sum(axis=0).argsort()[-n_features:][::-1]

# find index of 3 features with biggest negative shap impart
selected_negative_features = np.squeeze(shap_values_over_time[-1])[subj].sum(axis=0).argsort()[:n_features][::-1]

selected_features = np.concatenate((selected_positive_features, selected_negative_features))

fig1 = plt.figure(figsize=(15,5))
ax1 = fig1.add_subplot(121)
ax = sns.barplot(y=np.array(features)[selected_features], x=np.squeeze(shap_values_over_time[-1])[subj].sum(axis=0)[selected_features], palette="RdBu_r")
ax.title.set_text(f'SHAP values for subj {subj} ')

non_norm_subj_df = reverse_normalisation_for_subj(pd.DataFrame(data=test_X_np[subj], columns = features), normalisation_parameters_df)
median_norm_feature_df = non_norm_subj_df.median(axis=0)[selected_features]

ax2 = fig1.add_subplot(122)
font_size=12
bbox=[0, 0, 1, 1]
ax2.axis('off')
cell_text = []
for row in range(len(median_norm_feature_df)):
    cell_text.append([median_norm_feature_df.iloc[row].astype(str)])
mpl_table = ax2.table(cellText = cell_text, rowLabels = median_norm_feature_df.index, bbox=bbox, colLabels=['Normalised value'], cellLoc='center', colLoc='center', loc='center')
mpl_table.auto_set_font_size(False)
mpl_table.set_fontsize(font_size)
fig1.set_tight_layout(True)
# set figure title
fig1.suptitle(f'Explanation of prediction for subj {subj} with a probability of {outcome} of {subj_pred_over_ts[-1]*100:.2f}%', fontsize=20)

plt.show()

In [ ]:
if save_figures:
    fig1.savefig(os.path.join(out_dir, 'final_prediction.png'), dpi=600)

## Plot relevant features in relation to training population

In [ ]:
fig2 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features) // ncols + (len(selected_features) % ncols > 0)

# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    temp_pop_df = non_normalised_train_X_df[non_normalised_train_X_df.sample_label == features[feature]]
    sns.histplot(temp_pop_df.value, ax=ax)
    plt.scatter(median_norm_feature_df[features[feature]], 0, marker='o', s=500)
    if (n % ncols) == 1:
        if n <= len(selected_features) / 2:
            ax.set_title(r"$\bf{Positive\ features}$" +f'\n\n{features[feature]}')
        else:
            ax.set_title(r"$\bf{Negative\ features}$" + f'\n\n{features[feature]}')

    else:
        ax.set_title(features[feature])

plt.tight_layout()

In [ ]:
if save_figures:
    fig2.savefig(os.path.join(out_dir, 'features_histogram_comparison.png'), dpi=600)

## Plot evolution of prediction & explanation over time

In [ ]:
overall_prevailing_features = False

In [ ]:
cumulative_shap_values_over_time = np.array([shap_values_over_time[ts][0].sum(axis=1) for ts in range(n_time_steps)])

# find index of 3 features with biggest positive shap impart & index of 3 features with biggest negative shap impart
if overall_prevailing_features:
    # prevailing features over cumulative time
    selected_negative_features = cumulative_shap_values_over_time[:, subj].sum(axis=0).argsort()[:n_features][::-1]
    selected_positive_features = cumulative_shap_values_over_time[:, subj].sum(axis=0).argsort()[-n_features:][::-1]
else:
    # prevailing features at last timepoint
    selected_positive_features = np.squeeze(shap_values_over_time[-1])[subj].sum(axis=0).argsort()[-n_features:][::-1]
    selected_negative_features = np.squeeze(shap_values_over_time[-1])[subj].sum(axis=0).argsort()[:n_features][::-1]

selected_features = np.concatenate((selected_positive_features, selected_negative_features))

fig3 = plt.figure(figsize=(15,10))

k=1
alpha=0.3

timestep_axis = np.array(range(n_time_steps))

positive_color_palette = sns.color_palette("mako", n_colors=len(selected_positive_features))
negative_color_palette = sns.color_palette("flare_r", n_colors=len(selected_negative_features))

timestep_axis = np.array(range(n_time_steps))
ax = sns.lineplot(x=timestep_axis, y=subj_pred_over_ts, label='probability', linewidth = 2)


pos_baseline = subj_pred_over_ts
neg_baseline = subj_pred_over_ts
pos_count = 0
neg_count = 0
for i, feature in enumerate(selected_features):
    subj_cumulative_shap_value_over_time = cumulative_shap_values_over_time[:, subj, feature]
    positive_portion = (subj_cumulative_shap_value_over_time > 0)
    negative_portion = (subj_cumulative_shap_value_over_time < 0)

    pos_function = subj_cumulative_shap_value_over_time.copy()
    pos_function[negative_portion] = 0

    neg_function = subj_cumulative_shap_value_over_time.copy()
    neg_function[positive_portion] = 0

    if feature in selected_positive_features:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1

    positive_feature = pos_baseline + k * pos_function
    ax.fill_between(timestep_axis, pos_baseline, positive_feature, color=feature_color, alpha=alpha, label=features[feature])
    pos_baseline = positive_feature

    negative_feature = neg_baseline + k * neg_function
    ax.fill_between(timestep_axis, negative_feature, neg_baseline, color=feature_color, alpha=alpha)
    neg_baseline = negative_feature

ax.legend(fontsize='x-large')

ax.set_title(f'Predictions for subject {subj} of external validation set along time', fontsize=20)
ax.set_xlabel('Time from admission (hours)', fontsize=15)
ax.set_ylabel(f'Probability of {outcome}', fontsize=15)

plt.plot()



In [ ]:
if save_figures:
    fig3.savefig(os.path.join(out_dir, 'prediction_over_time.png'), dpi=600)

## Plot selected features over time

In [ ]:
fig4 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features) // ncols + (len(selected_features) % ncols > 0)

pos_count = 0
neg_count = 0
# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    if feature in selected_positive_features:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    sns.lineplot(y=features[feature], x='index', data=non_norm_subj_df.reset_index(), color=feature_color, ax=ax)

    if (n % ncols) == 1:
        if n <= len(selected_features) / 2:
            ax.set_title(r"$\bf{Positive\ features}$" +f'\n\n{features[feature]}')
        else:
            ax.set_title(r"$\bf{Negative\ features}$" + f'\n\n{features[feature]}')
    else:
        ax.set_title(features[feature])
plt.tight_layout()


In [ ]:
if save_figures:
    fig4.savefig(os.path.join(out_dir, 'features_over_time.png'), dpi=600)

## Plot contribution of a specific feature

In [ ]:
np.array(features)

In [ ]:
selected_features = ["median_mean_blood_pressure", "median_diastolic_blood_pressure", "median_systolic_blood_pressure"]


In [ ]:
selected_features_idx = [np.where(np.array(features) == selected_feature)[0][0] for selected_feature in selected_features]
selected_features_idx

In [ ]:
cumulative_shap_values_over_time = np.array([shap_values_over_time[ts][0].sum(axis=1) for ts in range(n_time_steps)])
subj_pred_over_ts = predictions_over_time[:,subj]

fig3 = plt.figure(figsize=(15,10))

k=1
alpha=0.3

timestep_axis = np.array(range(n_time_steps))

positive_color_palette = sns.color_palette("mako", n_colors=len(selected_positive_features))
negative_color_palette = sns.color_palette("flare_r", n_colors=len(selected_negative_features))

timestep_axis = np.array(range(n_time_steps))
ax = sns.lineplot(x=timestep_axis, y=subj_pred_over_ts, label='probability', linewidth = 2)
ax2 = ax.twinx()


pos_baseline = subj_pred_over_ts
neg_baseline = subj_pred_over_ts
pos_count, neg_count = 0, 0
for i, feature in enumerate(selected_features_idx):
    subj_cumulative_shap_value_over_time = cumulative_shap_values_over_time[:, subj, feature]
    positive_portion = (subj_cumulative_shap_value_over_time > 0)
    negative_portion = (subj_cumulative_shap_value_over_time < 0)

    pos_function = subj_cumulative_shap_value_over_time.copy()
    pos_function[negative_portion] = 0

    neg_function = subj_cumulative_shap_value_over_time.copy()
    neg_function[positive_portion] = 0

    if sum(subj_cumulative_shap_value_over_time) > 0:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1

    positive_feature = pos_baseline + k * pos_function
    ax.fill_between(timestep_axis, pos_baseline, positive_feature, color=feature_color, alpha=alpha, label=features[feature])
    pos_baseline = positive_feature

    negative_feature = neg_baseline + k * neg_function
    ax.fill_between(timestep_axis, negative_feature, neg_baseline, color=feature_color, alpha=alpha)
    neg_baseline = negative_feature

    sns.scatterplot(y=features[feature], x='index', data=non_norm_subj_df.reset_index(), ax=ax2, legend=False, color=feature_color)

ax.legend(fontsize='x-large')

ax.set_title(f'Predictions for subject {subj} of test set along time', fontsize=20)
ax.set_xlabel('Time from admission (hours)', fontsize=15)
ax.set_ylabel(f'Probability of {outcome}', fontsize=15)

plt.plot()

In [ ]:
fig4 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features_idx) // ncols + (len(selected_features_idx) % ncols > 0)

pos_count = 0
neg_count = 0
# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features_idx):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    if sum(subj_cumulative_shap_value_over_time) > 0:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    sns.lineplot(y=features[feature], x='index', data=non_norm_subj_df.reset_index(), color=feature_color, ax=ax)

    ax.set_title(features[feature])
plt.tight_layout()

In [ ]:
shap_values_over_time[-1][0].shape

In [ ]:
import shap

auto=False

X_test_2D = test_X_np.reshape(-1,n_channels)
shap_values_2D = shap_values_over_time[-1][0].reshape(-1,n_channels)
x_test_2d = pd.DataFrame(data=X_test_2D, columns = features)

for n, feature in enumerate(selected_features_idx):
    if auto == True:
        # automatic choice of interaction
        shap.dependence_plot(features[feature], shap_values_2D, x_test_2d)
    else:
        shap.dependence_plot(features[feature], shap_values_2D, x_test_2d, interaction_index="median_NIHSS")

## Finding interesting subjects

Big change in prediction:
- subj 58
- subj 95

Standard:
- subj 87

Visualize change in prediction from start to end of the 72h

In [ ]:
predictions_over_time[-1,:] - predictions_over_time[0,:]